In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import dependent libraries
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np


import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv('tags_ratings_data.csv')
df.head(2)

,Unnamed: 0,user_id,restaurant_id,Rating_x,title,No_of_Reviews,Tags,Details,Range_n_Cuisine,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes
0,0,0.0,363.0,0.0,Chartreuse Moose Cappuccino Bar & Bistro,170.0,"wraps , breakfast wrap , beans , great coffee...","{'CUISINES': 'Cafe, Canadian', 'Special Diets'...","['$$ - $$$', 'Quick Bites', 'Cafe', 'Canadian']",NaN,NaN,NaN,NaN
1,1,0.0,363.0,0.0,Chartreuse Moose Cappuccino Bar & Bistro,170.0,"wraps , breakfast wrap , beans , great coffee...","{'CUISINES': 'Cafe, Canadian', 'Special Diets'...","['$$ - $$$', 'Quick Bites', 'Cafe', 'Canadian']",NaN,NaN,NaN,NaN


In [5]:
df1 = df.copy()
df1 = df1.drop(['Unnamed: 0'], axis = 1)
df1.head(1)

,user_id,restaurant_id,Rating_x,title,No_of_Reviews,Tags,Details,Range_n_Cuisine,P_Tags,N_Tags,Pos_Review_Quotes,Neg_Review_Quotes
0,0.0,363.0,0.0,Chartreuse Moose Cappuccino Bar & Bistro,170.0,"wraps , breakfast wrap , beans , great coffee...","{'CUISINES': 'Cafe, Canadian', 'Special Diets'...","['$$ - $$$', 'Quick Bites', 'Cafe', 'Canadian']",NaN,NaN,NaN,NaN


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2784 entries, 0 to 2783
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            2784 non-null   float64
 1   restaurant_id      2784 non-null   float64
 2   Rating_x           2784 non-null   float64
 3   title              2784 non-null   object 
 4   No_of_Reviews      2532 non-null   float64
 5   Tags               214 non-null    object 
 6   Details            2784 non-null   object 
 7   Range_n_Cuisine    2784 non-null   object 
 8   P_Tags             1808 non-null   object 
 9   N_Tags             1778 non-null   object 
 10  Pos_Review_Quotes  1801 non-null   object 
 11  Neg_Review_Quotes  1221 non-null   object 
dtypes: float64(4), object(8)
memory usage: 261.1+ KB


In [7]:
df1.Rating_x.value_counts()

5.0    554
4.5    323
2.5    318
4.0    284
1.0    278
2.0    266
0.5    221
3.0    216
3.5    153
1.5    124
0.0     47
Name: Rating_x, dtype: int64

In [8]:
for col in df1.columns:
    df1[col] = [0.5 if x == 0 else x for x in df1[col]]
df1.Rating_x.value_counts()

5.0    554
4.5    323
2.5    318
4.0    284
1.0    278
0.5    268
2.0    266
3.0    216
3.5    153
1.5    124
Name: Rating_x, dtype: int64

In [9]:
df1.No_of_Reviews.unique()

array([170.,  13.,  22.,  10.,  17.,  56.,  25.,   6.,   9.,   7.,   5.,
        19.,   4.,  23.,  nan,  16.,   3.,  74.,  20.,   2.,  11.,  28.,
        43.,  24.,  12.,   1.,  64.,  14.,  27.,   8.,  35.,  29.,  33.,
        15.,  30.,  50.,  36.,  48., 105.,  26.,  47.,  18., 248., 143.,
       104.,  58.,  34.,  44.,  40.,  54.,  68.])

In [10]:
df1.No_of_Reviews.isnull().sum()

252

In [11]:
df1.No_of_Reviews = df1.No_of_Reviews.fillna(method = 'ffill')
df1.No_of_Reviews.isnull().sum()

0

In [12]:
df1["Cuisine"]=df1.Details.str.split("'CUISINES':").str[1]
df1.Cuisine = df1.Cuisine.astype(str)
df1["Cuisine"]=df1.Cuisine.apply(lambda x: x.replace('}','').replace("'",''))
df1["Cuisine"]=df1.Cuisine.str.split(", Special Diets:").str[0]
df1["Cuisine"]=df1.Cuisine.str.split(", Meals:").str[0]
df1["Cuisine"]=df1.Cuisine.str.split(", FEATURES:").str[0]
df1.Cuisine.value_counts()

nan                                           620
 Fast food                                    475
 Fast food, Canadian                          382
 American, Fast food                          146
 Deli                                         112
 Fast food, Canadian, Cafe                     84
 Fast food, American, Canadian                 82
 American, Fast food, Canadian                 80
 Cafe, Fast food, Canadian                     78
 Cafe, Canadian                                75
 Canadian                                      73
 American, Deli                                72
 American                                      70
 American, Fast food, Deli                     64
 Canadian, Fast food                           63
 Cafe                                          48
 Cafe, Fast food                               27
 Street Food, Fast food                        24
 Fast food, American                           24
 Fast food, Canadian, American                 22


In [13]:
df1.Cuisine = df1.Cuisine.fillna('not_availabe')
df1.P_Tags = df1.P_Tags.fillna('not_availabe')
df1.N_Tags = df1.N_Tags.fillna('not_availabe')
df1.Pos_Review_Quotes = df1.Pos_Review_Quotes.fillna('not_availabe')
df1.Neg_Review_Quotes = df1.Neg_Review_Quotes.fillna('not_availabe')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2784 entries, 0 to 2783
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            2784 non-null   float64
 1   restaurant_id      2784 non-null   float64
 2   Rating_x           2784 non-null   float64
 3   title              2784 non-null   object 
 4   No_of_Reviews      2784 non-null   float64
 5   Tags               214 non-null    object 
 6   Details            2784 non-null   object 
 7   Range_n_Cuisine    2784 non-null   object 
 8   P_Tags             2784 non-null   object 
 9   N_Tags             2784 non-null   object 
 10  Pos_Review_Quotes  2784 non-null   object 
 11  Neg_Review_Quotes  2784 non-null   object 
 12  Cuisine            2784 non-null   object 
dtypes: float64(4), object(9)
memory usage: 282.9+ KB


In [14]:
df2 = df1.copy()
df2 = pd.get_dummies(df2, columns = ['P_Tags','N_Tags','Pos_Review_Quotes','Neg_Review_Quotes','Cuisine'])
df2.fillna(0)
df2.shape

(2784, 87)

In [15]:
df2.head(2)

,user_id,restaurant_id,Rating_x,title,No_of_Reviews,Tags,Details,Range_n_Cuisine,"P_Tags_best pizza,more pizza,order excellent,friendly staff food,more loaded side,great pizza,awesome pizza,crust awesome place,really good pizza,great crust sauce,good service,great thing,true place,great value,great dinner,many thing,quick service,good size,nice server","P_Tags_best subway,good food,good service,healthy food subway,great job subway,good place,good sub u,many subway,good value,good job location,best server,good variety,good value quality,great food,great service,good old way,good chain store,good amount filling,friendly service quality,friendly staff dinner","P_Tags_food amazing food,good thing,good way,friendly service,nice upgrade building,bar charming bartender,one best pizza,fantastic meal","P_Tags_friendly tried place,good atmosphere,total great place,huge portion,good service,great decor waitress,great value,more item menu,deluxe burger,very clean pub","P_Tags_good coffee,good getting coffee,good cup coffee,good service,good food,box good service,great coffee,good breakfast,good place,fast good place,quick coffee,coffee good staff,great fast service,great service,good location,nice coffee,coffee more parking,coffee excellent,good change,good menu","P_Tags_good food,good selection,great service staff","P_Tags_good mexican food,restaurant great food,good meal,more time,such great experience,very good restaurant,friendly server,my favorite restaurant,their best employee,lovely dinner,much better hubby","P_Tags_great breakfast,good value,great food,great service,friendly staff husband,mile fabulous program,delicious egg benny,reliably good coffee,excellent atmosphere,good service,good food,good food place,good quick service,good food skip,good place,good restaurant,great service,good meal,good restaurant lunch,great food","P_Tags_love sushi,sushi nice,fresh friendly service,quick food,delicious food,authentic wrapped sushi,wonderful food,great service,great place,many comment,huge party platter,still good wasabi,absolutely delicious thing",P_Tags_not_availabe,"P_Tags_perfect food,many more dinner,wonderful delicious buffet,nice variety,our favorite server,our friendly server,our favorite place,u quick meal","P_Tags_ready fish excellent,friendly parking,good care,good option owner,very friendly fish,coleslaw delicious,excellent food,minute safe bite,great love","N_Tags_long line order,few tim horton,wrong order,punched wrong order,long line,long line ups,few staff,parking small lot,least staff,sour cream donut,world limited menu,small discount,made rude comment,small drink approx,wet dripping hand,incredibly poor service,very slow service,worst ever experience,vulnerable finance,ice cold food",N_Tags_not_availabe,"N_Tags_poor service,food cold return,food nasty,service bland food,cold waitress,total waste money,cold tea,absolutely worst service,long hour,one few place,just bad pot","N_Tags_subway stale bread,bad sandwich,cold wrap,single bottled beverage,refill cold refreshment,few day,serious lesson,little cardboard piece,tape chicken,very filthy staff,even single portion,not sure problem,late one night,approximately pm,not right temperature,worst subway,course few week,last few year,partial uniform,generally dirty woman","N_Tags_supper bad decision,really awful pizza,area dirty table,terrible place",N_Tags_terrible smog price,"N_Tags_tired old barman,tired old bar","N_Tags_tough end potato,bad time,not many patron","Pos_Review_Quotes_a great latte,spacious place with reasonable selection of goodies great staff but torture device chairs,a nice spot to hangout for a coffee,such good coffee and lemon tarts","Pos_Review_Quotes_a great stay,delicious grilled cheese ,the fairmont made our christmas dinner special ,wonderful stay,great return stay,love this hotel,absolutely stunning over the top beautiful experience,magical banff,best stay yet ,exceptional service,beautiful place even if you end up in the carriage house,beautiful fo

In [16]:
df2.restaurant_id.value_counts()

10.0     1176
217.0     638
41.0      510
225.0     266
204.0      64
363.0      54
367.0      18
368.0      14
370.0      10
376.0       7
373.0       6
380.0       6
379.0       6
369.0       5
364.0       4
Name: restaurant_id, dtype: int64

In [17]:
#model establishment
from lightfm.data import Dataset
dataset = Dataset()
dataset.fit(df2.user_id,df2.restaurant_id)
type(dataset)
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 20, num_items 15.


In [18]:
from lightfm import LightFM
dataset.fit_partial(items=df2.restaurant_id,
                    item_features=['Rating_x'])
dataset.fit_partial(items=df2.restaurant_id,
                    item_features=['No_of_Reviews'])
tar_cols = [x for x in df2.columns[8:]]
dataset.fit_partial(items = df2.restaurant_id,
                   item_features = tar_cols)   
# Check data
print(type(dataset))
print(dataset.model_dimensions())
print(dataset.item_features_shape())
print(dataset.interactions_shape())

<class 'lightfm.data.Dataset'>
(20, 96)
(15, 96)
(20, 15)


In [19]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [20]:
# Creating interaction matrix using rating data
interactions = create_interaction_matrix(df = df2,
                                         user_col = 'user_id',
                                         item_col = 'restaurant_id',
                                         rating_col = 'Rating_x')
interactions.head(2)

restaurant_id,10.0,41.0,204.0,217.0,225.0,363.0,364.0,367.0,368.0,369.0,370.0,373.0,376.0,379.0,380.0
user_id,,,,,,,,,,,,,,,
0.5,0.0,0.0,0.0,145.0,0.0,3.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
1.0,0.0,0.0,20.0,0.0,19.0,0.0,0.0,0.0,7.0,0.0,1.5,0.0,4.5,0.0,0.0


In [21]:
from scipy.sparse import csr_matrix
interactions = csr_matrix(interactions.values)

In [22]:
def build_dict(df,tar_cols,val_list):
    rst = {}
    for col in tar_cols:
        rst[col] = df[col]
    sum_val = sum(list(rst.values())) # get sum of all the tfidf values
    
    if(sum_val == 0):
        return rst
    else:
        
        w = (2-sum(val_list))/sum_val # weight for each tag to be able to sum to 1
        for key,value in rst.items():
            rst[key] = value * w
    return rst

In [23]:
# get max of each column to regularize value to [0,1]
max_rating = max(df2.Rating_x)
max_b_no_rev = max(df2.No_of_Reviews)
print(max_rating)
print(max_b_no_rev)

5.0
248.0


In [24]:
# give CF info weight 0.5, all other 0.5. Then in others, give (star, review count) 0.25 and tags 0.25
item_features = dataset.build_item_features(((x['restaurant_id'], 
                                              {'Rating_x':0.5*x['Rating_x']/max_rating,
                                               'No_of_Reviews':0.5*x['No_of_Reviews']/max_b_no_rev,
                                               **build_dict(x,tar_cols,[0.5*x['Rating_x']/max_rating,
                                                           0.5*x['No_of_Reviews']/max_b_no_rev])})
                                              for index,x in df2.iterrows()))
print(repr(item_features))
print(item_features.shape)

<15x96 sparse matrix of type '<class 'numpy.float32'>'
	with 1230 stored elements in Compressed Sparse Row format>
(15, 96)


In [25]:
# check features to see if weights make sense
idx = 10
tt = list(item_features[idx].nonzero())
print(item_features[idx].shape)
print(tt)

(1, 96)
[array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), array([10, 15, 16, 19, 30, 38, 51, 65], dtype=int32)]


In [26]:
tt = item_features[idx].todense()
import pandas as pd
tt = pd.DataFrame(tt)
print(list(map(set,tt.values)))# feature weight sum to 1, so if two are nonzero, each take weight 0.5. 

[{0.0, 0.03448276, 0.16268632, 0.14137931, 0.010706339, 0.16268633}]


In [27]:
#train-test split
seed = 1
from lightfm import cross_validation 
train,test=cross_validation.random_train_test_split(interactions, test_percentage=0.5,
                                                    random_state=np.random.RandomState(seed))

print('The dataset has %s users and %s items, '
      'with %s interactions in the test and %s interactions in the training set.'
      % (train.shape[0], train.shape[1], test.getnnz(), train.getnnz()))

train.multiply(test).nnz == 0 # make sure train and test are truly disjoint

The dataset has 20 users and 15 items, with 46 interactions in the test and 46 interactions in the training set.


True

In [28]:
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k,recall_at_k

In [29]:
NUM_THREADS = 25
NUM_COMPONENTS = 21    
NUM_EPOCHS = 16
ITEM_ALPHA = 5.97967e-6
learning_rate=0.033
k =5
## Pure Collaborative Filtering models
# Let's fit a WARP model
model = LightFM(loss='warp',random_state=seed,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS,
               learning_rate=learning_rate)

#  time it.
%time model = model.fit(train,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)



# Compute and print the AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

test_auc = auc_score(model, test,num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)


print("Train precision: %.4f" % precision_at_k(model, train, k=k,num_threads=NUM_THREADS).mean())
print("Test precision: %.4f" % precision_at_k(model, test,train_interactions=train,
                                              k=k,num_threads=NUM_THREADS).mean())

CPU times: user 14.1 ms, sys: 2.32 ms, total: 16.4 ms
Wall time: 16.8 ms
Collaborative filtering train AUC: 0.90031767
Collaborative filtering test AUC: 0.4537261
Train precision: 0.3900
Test precision: 0.1895


In [30]:
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.indices)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [31]:
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [32]:
# Create User Dict
user_dict = create_user_dict(interactions=interactions)
# Create Item dict
restr_dict = create_item_dict(df = df1,
                               id_col = 'restaurant_id',
                               name_col = 'title')

In [33]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
                                 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [64]:
# ## Calling 5 restaurant recommendation for user id 7
# rec_list = sample_recommendation_user(model, 
#                                       interactions, 
#                                       user_id = 7, 
#                                       item_dict = restr_dict,
#                                      user_dict=user_dict,
#                                       threshold=3,
#                                      show=True)
sample_recommendation_user(model, interactions, 
                           user_id='ff52b7331f2ccab0582678644fed9d85', 
                           user_dict=user_dict, 
                           item_dict=restr_dict,
                          threshold=3)

KeyError: 'ff52b7331f2ccab0582678644fed9d85'

In [44]:
print(rec_list)

NameError: name 'rec_list' is not defined